### `tes-thermo`
---

In this notebook, we will use `tes-thermo` to calculate the equilibrium compositions for the steam methane reforming (SMR) process.

Version 0.1.1 of `tes-thermo` allows the user to leverage the `thermo` library to retrieve information about the components and also enables the addition of new components.

First, import the Component and Gibbs classes. Then, specify the components to be considered.

In [ ]:
%pip install -qU tes-thermo

In [1]:
%cd ..

/Users/mitoura/Desktop/Repositories/tes-thermo


In [2]:
from tes_thermo.utils import Component
from tes_thermo.gibbs import Gibbs
import numpy as np

new_components = {
        "methane": {
            "name": "methane",
            "Tc": 190.6, "Tc_unit": "K",
            "Pc": 45.99, "Pc_unit": "bar",
            "omega": 0.012,
            "Vc": 98.6, "Vc_unit": "cm³/mol",
            "Zc": 0.286,
            "Hfgm": -74520, "Hfgm_unit": "J/mol",
            "Gfgm": -50460, "Gfgm_unit": "J/mol",
            "structure": {"C": 1, "H": 4},
            "phase": "g",
#            "kijs": [0, 0, 0, 0, 0, 0], # Interaction parameters with other components
            "cp_polynomial": lambda T: 8.314 * (1.702 + 0.009081* T -0.000002164*T**2),
        }
    }

components = ['water','carbon monoxide', 'carbon dioxide', 'hydrogen', 'methanol', 'C12H24O2','143-07-7', 'C11H24', 'C12H26']

In `new_components`, the components to be added are defined. In this case, the user must specify all the thermodynamic properties of the component as well as the polynomial used to calculate $C_p$.

For this example, the following polynomial was used:

$$C_p(T) = R \times \left( 1.702 + 0.009081T - 0.000002164T^2 \right)$$

where $T$ is the temperature in Kelvin and $ C_p $ is the heat capacity in J/(mol·K).

The components listed in `components` will be instantiated using the `Chemical` class from the `thermo` library.

In [15]:
comps = Component(components, new_components)
comps = comps.get_components()
gibbs = Gibbs(components=comps,
              equation='Virial', 
              inhibited_component = ['carbon dioxide','carbon monoxide', 'methane'])

res = gibbs.solve_gibbs(T=573, T_unit= 'K',
                         P=30, P_unit='bar',
                         # 'water (1)','carbon monoxide (2)', 'carbon dioxide (3)', 'hydrogen (4)', 'methanol (5)', 'C12H24O2 (6)', 'Lauric acid (7)', 'C11H24 (8)', 'C12H26 (9)', 'methane (10)'
                         #                (1) (2) (3) (4) (5) (6) (7) (8) (9)
                         initial=np.array([0,  0,  0,  1,  0,  0, 1,  0,  0,  0]))

2025-08-15 11:46:45 - tes_thermo.gibbs.gibbs - INFO - Initializing Gibbs class...
2025-08-15 11:46:45 - tes_thermo.gibbs.gibbs - INFO - Components created as Chemical from the thermo library: ['water', 'carbon monoxide', 'carbon dioxide', 'hydrogen', 'methanol', 'C12H24O2', '143-07-7', 'C11H24', 'C12H26']
2025-08-15 11:46:45 - tes_thermo.gibbs.gibbs - INFO - Manually defined components: [{'name': 'methane', 'Tc': 190.6, 'Pc': 4599000.0, 'omega': 0.012, 'Vc': 9.859999999999998e-05, 'Zc': 0.286, 'Hfgm': -74520.0, 'Gfgm': -50460.0, 'phase': 'g', 'structure': {'C': 1, 'H': 4}, 'kijs': None, 'cp_coeffs': None, 'cp_polynomial': <function <lambda> at 0x10b35a340>}]
2025-08-15 11:46:45 - tes_thermo.gibbs.gibbs - INFO - Inhibited components: ['carbon dioxide', 'carbon monoxide', 'methane']
2025-08-15 11:46:45 - tes_thermo.gibbs.gibbs - INFO - Equation of state: Virial
2025-08-15 11:46:45 - tes_thermo.gibbs.gibbs - INFO - Calculated kijs for methane: [0.013270068977719762, 0.00018971524403865736

[-219763.84381529444, -164683.0497975347, -399372.5767152574, -2906.8202366709284, -131744.4448241235, 10953.213266401784, -31743.65171318903, 286788.49857666437, 320272.3530190346, -32274.842968364843]


2025-08-15 11:46:45 - tes_thermo.gibbs.gibbs - INFO - Optimal solution found successfully


Note that the `Component` class takes both `components` and `new_components` as arguments. The `get_components` method already returns the required structure to instantiate the `Gibbs` class.

With this information, the `solve_gibbs` method returns the equilibrium compositions at temperature $T$ and pressure $P$, given the initial compositions specified by `initial`.

In [16]:
res

{'Temperature (K)': 573.0,
 'Pressure (bar)': 30.000000000000004,
 'Water': 0.6515359964353197,
 'Carbon monoxide': 1.0009841923055985e-05,
 'Carbon dioxide': 1.0009909959585808e-05,
 'Hydrogen': 0.0013846055266460025,
 'Methanol': 2.483250389545864e-06,
 'Decyl acetate': 4.054035765386051e-06,
 'Lauric acid': 0.6742117062641858,
 'Undecane': 0.2556680331136172,
 'Dodecane': 0.09141915579007258,
 'Methane': 1.0009878910735158e-05}